<h1 align="center">Data Cleaning - <a href="https://www.kaggle.com/puneeshk/lending-loan-club-dataset" target=_blank><u>Lending Club Dataset</u></a>
</h1>


# Loading the libraries

In [219]:
import pandas as pd
import numpy as np
from copy import deepcopy
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)



# Loading the dataset

In [220]:
# load the big dataset into a pandas dataframe
big_df = pd.read_csv("loan.csv", index_col=False, dtype='unicode')

# have a quick glance at the dataframe
big_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


# Choosing only those columns that's relevant for analysis and visualization

My goal is to help Lending Club team gain insight on what consitutes a 'Good Loans'and what constitutes a 'Bad Loans'. Good Loans are those loans with status of "Current", "Issued" and "Fully Paid" in the <font color='purple'>loan_status</font> column. Bad Loans are those loans with the status other than "Current", "Issued" and "Fully Paid".

<font color='navy'>Particularly, I plan to answer the following questions for the Lending Club team:</font>
- What proportion of loans are 'Good Loans' based on term?
- What proportion of loans are 'Good Loans' based on grade?
- Is 'Good Loans' somehow linked with the job of the borrowers?
- Is 'Good Loans' somehow linked with the job duration of the borrowers?
- What are the most frequent purpose values for 'Bad Loans'?

<font color='navy'> Hence, the relevant variables for my theme, I believe, are:</font>

<font color='purple'>id</font>: A unique LC assigned ID for the loan listing.
- The primary key for the dataset.

<font color='purple'>term</font>: The number of payments on the loan. Values are in months and can be either 36 or 60.
- To check which loan term are more prone to default.
  
<font color='purple'>grade</font>: LC assigned loan grade.
- To check if grade is directly correlated with loan status.
  
<font color='purple'>purpose</font>: A category provided by the borrower for the loan request.
- To identify frequent purpose values for 'Bad Loans'.
  
<font color='purple'>emp_title</font>: The job title supplied by the Borrower when applying for the loan.
- To identify job characteristics of the borrowers that fall in 'Good Loans' category.

<font color='purple'>emp_length</font>: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
- To identify if there's some pattern in employment duration and 'Good Loans'.

<font color='purple'>loan_status</font>: Current status of the loan. 
- To create loan_type column out of this column which categorize each entry as 'Good Loans' or 'Bad Loans'.

 



In [221]:
# important variable to consider
imp_var = [
    'id',
    'term', 
    'grade', 
    'emp_title', 
    'emp_length', 
    'purpose',
    'loan_status'
]

# pandas dataframe consisting of important variables
df = big_df[imp_var]

# have a quick glance at the new dataframe
df.head(10)

,id,term,grade,emp_title,emp_length,purpose,loan_status
0,1077501,36 months,B,NaN,10+ years,credit_card,Fully Paid
1,1077430,60 months,C,Ryder,< 1 year,car,Charged Off
2,1077175,36 months,C,NaN,10+ years,small_business,Fully Paid
3,1076863,36 months,C,AIR RESOURCES BOARD,10+ years,other,Fully Paid
4,1075358,60 months,B,University Medical Group,1 year,other,Current
5,1075269,36 months,A,Veolia Transportaton,3 years,wedding,Fully Paid
6,1069639,60 months,C,Southern Star Photography,8 years,debt_consolidation,Fully Paid
7,1072053,36 months,E,MKC Accounting,9 years,car,Fully Paid
8,1071795,60 months,F,NaN,4 years,small_business,Charged Off
9,1071570,60 months,B,Starbucks,< 1 year,other,Charged Off


# Data Cleaning

Check if any of these columns have missing value.



In [222]:
def check_missing_value(arr):
    for var in arr:
        if df[var].isnull().values.any() == True:
            print(f'The column {var} has missing values.')
    print('\n')

check_missing_value(imp_var)

The column emp_title has missing values.
The column emp_length has missing values.





Cleaning <font color='purple'>emp_length</font> column by removing all the strings and converting employment length to integer. Missing values are filled with -1 value. The rows with the value '< 1 year' is converted to 0 and '10+ years' is converted to 10. This step is mainly done to make it easier to sort the column by years for visualization purpose.  After they are sorted, their original name is restored. Finally, renaming it as <font color='purple'>emp_length_years</font> to make it easier to identify the unit for this column.


In [232]:
# helper function to build df
def build_df(var_name):

    # group the data based on var_name and store it as a dictionary
    data = dict(df[var_name].value_counts())

    # convert dictionary to pandas dataframe
    data = pd.DataFrame(data.items(),columns = [var_name,'count'])
    
    return data

# rename emp_length to emp_length_years
df.rename(columns = {'emp_length':'emp_length_years'}, inplace = True)

# remove years from row values and make '< 1 year' as 0 and 
# '10+ years' as 10.
df.replace({'emp_length_years': 
    {'< 1 year':0,
    '1 year':1, 
    '2 years':2,
    '3 years':3, 
    '4 years':4, 
    '5 years':5, 
    '6 years':6,
    '7 years':7,
    '8 years':8,
    '9 years':9,
    '10+ years':10
    }
    }, inplace=True)

# Handling missing values by replacing it with -1 
if df['emp_length_years'].isnull().values.any() == True:
    df['emp_length_years'].fillna(-1, inplace=True) 

# convert the column dtype from float to int
df = df.copy(deep=True).astype({'emp_length_years': 'int64'})

# group data by emp_length_years
emp_df = build_df('emp_length_years').sort_values(by=['emp_length_years']).reset_index(drop=True)


# take a glance on the update
print('------------------------\n\
-1 denotes missing values\n \
0 denotes less than 1 year\n \
10 denotes 10 or more years\n \
------------------------')
emp_df.style.hide_index()



------------------------
-1 denotes missing values
 0 denotes less than 1 year
 10 denotes 10 or more years
 ------------------------


emp_length_years,count
-1,1075
0,4583
1,3240
2,4388
3,4095
4,3436
5,3282
6,2229
7,1773
8,1479


Now, we move on to clean <font color='purple'>emp_title</font> column. Since this column is derived from an open ended question, it is not really a categorical column. However, we could still derive some insights from this column. The missing values will need to be replaced with something like 'employer not mentioned', and we need to lowercase all the datapoints and remove extra whitespaces.

In [224]:
# lowercase all the data points in the column
df['emp_title'] = df['emp_title'].str.lower()

# remove all extra whitespaces
df['emp_title'] = df['emp_title'].str.strip()

# replace NaN missing values with 'employer not mentioned'
df.fillna('employer not mentioned', inplace=True)

# glance at the value count of the modified column
print("Borrower's top 20 employers (including the count of missing values):\n")
print(df['emp_title'].value_counts()[:21])
print('\n\n')

Borrower's top 20 employers (including the count of missing values):

employer not mentioned    2459
us army                    212
bank of america            138
at&t                        83
walmart                     82
wells fargo                 71
ibm                         69
kaiser permanente           69
verizon wireless            65
ups                         63
self                        60
usaf                        58
usps                        58
us air force                57
self employed               57
walgreens                   53
state of california         49
us navy                     49
lockheed martin             49
home depot                  47
us postal service           45
Name: emp_title, dtype: int64





Create <font color='purple'>loan_type</font> column that categorize each entry either 'Good Loans' or 'Bad Loans'. Loans with status of 'Fully Paid', 'Issued', or 'Current' as given 'Good Loans' value. The rest of the columns are given 'Bad Loans' value.

In [229]:
# loans with status of 'Fully Paid', 'Issued', or 'Current' as given 'Good Loans' value in the loan_type column
df.loc[((df['loan_status'] == 'Fully Paid') | (df['loan_status'] == 'Issued') | (df['loan_status'] == 'Current')), 'loan_type'] = 'Good Loans'

# the rest of the columns are given 'Bad Loans' value.
df['loan_type'] = df['loan_type'].fillna('Bad Loans')

# glance at the dataset
df.head(10)

,id,term,grade,emp_title,emp_length_years,purpose,loan_status,loan_type
0,1077501,36 months,B,employer not mentioned,10,credit_card,Fully Paid,Good Loans
1,1077430,60 months,C,ryder,0,car,Charged Off,Bad Loans
2,1077175,36 months,C,employer not mentioned,10,small_business,Fully Paid,Good Loans
3,1076863,36 months,C,air resources board,10,other,Fully Paid,Good Loans
4,1075358,60 months,B,university medical group,1,other,Current,Good Loans
5,1075269,36 months,A,veolia transportaton,3,wedding,Fully Paid,Good Loans
6,1069639,60 months,C,southern star photography,8,debt_consolidation,Fully Paid,Good Loans
7,1072053,36 months,E,mkc accounting,9,car,Fully Paid,Good Loans
8,1071795,60 months,F,employer not mentioned,4,small_business,Charged Off,Bad Loans
9,1071570,60 months,B,starbucks,0,other,Charged Off,Bad Loans


Saving the final dataframe as a CSV and Excel file

In [233]:
# saving the dataframe in a CSV format.
df.to_csv('lending_club_modified_data.csv', encoding='utf-8', index=False)

#savig the dataframe in an excel format.
with pd.ExcelWriter('lending_club_modified_data.xlsx') as writer:
    df.to_excel(writer)

# Now we are all set to move on to our analysis and visualization step. Let's go!